# IBM Instance Setup

In [1]:
%cd ..

/Users/uribagi/Documents/GitHub/Latent-IQP


In [2]:
from dotenv import load_dotenv
import os
import jax
import jax.numpy as jnp
import iqpopt as iqp
from iqpopt.utils import initialize_from_data, local_gates
import iqpopt.gen_qml as genq
from iqpopt.gen_qml.utils import median_heuristic
from utils.nisq import aachen_connectivity, efficient_connectivity_gates
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session
import pennylane as qml
from datasets.bipartites import BipartiteGraphDataset
from datasets.er import ErdosRenyiGraphDataset
import numpy as np

key = jax.random.PRNGKey(42)

# Experiments Poster

In [3]:
def load_ds(nodes, type, connectivity):
    ds_path = f'./datasets/raw_data/{nodes}N_{type}_{connectivity}.pkl'
    dataset = jnp.array(BipartiteGraphDataset(nodes = 1, edge_prob=0.1).from_file(ds_path).vectors.copy())
    return dataset

In [4]:
type = "ER"
conn = "Sparse"
ER10N = load_ds(10, type, conn)
ER14N = load_ds(14, type, conn)
ER18N = load_ds(18, type, conn)

[Dataset] Loaded 500 samples from ./datasets/raw_data/10N_ER_Sparse.pkl
  Created: 2025-05-30T13:09:11.886279
  Unique graphs: 500
  Version: 1.3
[Dataset] Loaded 1000 samples from ./datasets/raw_data/14N_ER_Sparse.pkl
  Created: 2025-05-30T13:09:28.785584
  Unique graphs: 1000
  Version: 1.3
[Dataset] Loaded 1000 samples from ./datasets/raw_data/18N_ER_Sparse.pkl
  Created: 2025-05-30T13:09:53.812165
  Unique graphs: 1000
  Version: 1.3


In [5]:
type = "Bipartite"

conn = "Sparse"
BP10NS = load_ds(10, type, conn)
BP14NS = load_ds(14, type, conn)
BP18NS = load_ds(18, type, conn)
conn = "Dense"
BP10ND = load_ds(10, type, conn)
BP14ND = load_ds(14, type, conn)
BP18ND = load_ds(18, type, conn)

[Dataset] Loaded 473 samples from ./datasets/raw_data/10N_Bipartite_Sparse.pkl
  Created: 2025-05-30T13:15:39.349125
  Unique graphs: 473
  Version: 1.0
[Dataset] Loaded 995 samples from ./datasets/raw_data/14N_Bipartite_Sparse.pkl
  Created: 2025-05-30T13:16:01.302922
  Unique graphs: 995
  Version: 1.0
[Dataset] Loaded 992 samples from ./datasets/raw_data/18N_Bipartite_Sparse.pkl
  Created: 2025-05-30T13:16:36.761074
  Unique graphs: 992
  Version: 1.0
[Dataset] Loaded 498 samples from ./datasets/raw_data/10N_Bipartite_Dense.pkl
  Created: 2025-05-30T13:15:42.038328
  Unique graphs: 498
  Version: 1.0
[Dataset] Loaded 995 samples from ./datasets/raw_data/14N_Bipartite_Dense.pkl
  Created: 2025-05-30T13:16:04.284752
  Unique graphs: 995
  Version: 1.0
[Dataset] Loaded 995 samples from ./datasets/raw_data/18N_Bipartite_Dense.pkl
  Created: 2025-05-30T13:17:03.846539
  Unique graphs: 995
  Version: 1.0


In [6]:
ER10NParams = np.load("results/params/params_10N_ER_Sparse_LR0.006838364713689533_SIGMA0.734109035909562_INIT0.400113391305837_NUMLAYERS2.npy")
ER14NParams = np.load("results/params/params_14N_ER_Sparse_LR0.02035688240275493_SIGMA0.8146109293590678_INIT1.3171747398920903_NUMLAYERS1.npy")
ER18NParams = np.load("results/params/params_18N_ER_Sparse_LR0.005956763927368095_SIGMA0.8221295744511952_INIT0.5859381161075201_NUMLAYERS2.npy")

In [7]:
BP10NSParams = np.load("results/params/params_10N_Bipartite_Sparse_LR0.02795366237712352_SIGMA0.9808444814012455_INIT0.765609221123329_NUMLAYERS1.npy")
BP14NSParams = np.load("results/params/params_14N_Bipartite_Sparse_LR0.006907903977137845_SIGMA1.0466739240034715_INIT0.8796771395628953_NUMLAYERS1.npy")
BP18NSParams = np.load("results/params/params_18N_Bipartite_Sparse_LR0.006312967697938845_SIGMA0.8081435463552646_INIT0.6667394152784892_NUMLAYERS2.npy")
BP10NDParams = np.load("results/params/params_10N_Bipartite_Dense_LR3.180387390187506e-05_SIGMA0.10532104404374365_INIT1.2714724852677064_NUMLAYERS1.npy")
BP14NDParams = np.load("results/params/params_14N_Bipartite_Dense_LR0.0008949151281083548_SIGMA0.10084894279938172_INIT1.5842054712672855_NUMLAYERS2.npy")
BP18NDParams = np.load("results/params/params_18N_Bipartite_Dense_LR0.00043865712569417307_SIGMA0.10245036558677464_INIT3.078356101278033_NUMLAYERS1.npy")

# Hardware test

In [8]:
load_dotenv('.env')

ibm_token = os.getenv('IBM_TOKEN')
instance = os.getenv("INSTANCE")
setup = True 

if setup:
    QiskitRuntimeService.save_account(channel="ibm_quantum", token=ibm_token, overwrite=True)

service = QiskitRuntimeService(channel="ibm_cloud", token = ibm_token, instance=instance)
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=156)
print(backend)

<IBMBackend('ibm_aachen')>


In [9]:
dev = qml.device('qiskit.remote', 
                    wires=backend.num_qubits, 
                    backend=backend, 
                    shots=512,
                    optimization_level=3)

## 10 Node Experiments

In [10]:
QUBITS = 10 * 9 // 2
grid_conn = aachen_connectivity()
gates_single_layer = efficient_connectivity_gates(grid_conn, QUBITS, 1)
gates_two_layer = efficient_connectivity_gates(grid_conn, QUBITS, 2)

IQP_single = iqp.IqpSimulator(QUBITS, gates_single_layer, device="lightning.qubit")
IQP_double = iqp.IqpSimulator(QUBITS, gates_two_layer, device="lightning.qubit")

@qml.qnode(dev)
def sample_single(trained_params):
    IQP_single.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def sample_double(trained_params):
    IQP_double.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

In [11]:
with Session(backend=backend, max_time="2h") as session:
    ER10NRes  = sample_double(ER10NParams)
    BP10NDRes = sample_single(BP10NDParams)
    BP10NSRes = sample_single(BP10NSParams)

## 14 Node Experiments

In [12]:
QUBITS = 14 * 13 // 2
grid_conn = aachen_connectivity()
gates_single_layer = efficient_connectivity_gates(grid_conn, QUBITS, 1)
gates_two_layer = efficient_connectivity_gates(grid_conn, QUBITS, 2)

IQP_single = iqp.IqpSimulator(QUBITS, gates_single_layer, device="lightning.qubit")
IQP_double = iqp.IqpSimulator(QUBITS, gates_two_layer, device="lightning.qubit")

@qml.qnode(dev)
def sample_single(trained_params):
    IQP_single.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def sample_double(trained_params):
    IQP_double.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

In [13]:
with Session(backend=backend, max_time="2h") as session:
    ER14NRes  = sample_single(ER14NParams)
    BP14NDRes = sample_single(BP14NDParams)
    BP14NSRes = sample_double(BP14NSParams)

## 18 Node Experiments

In [14]:
QUBITS = 18 * 17 // 2
grid_conn = aachen_connectivity()
gates_single_layer = efficient_connectivity_gates(grid_conn, QUBITS, 1)
gates_two_layer = efficient_connectivity_gates(grid_conn, QUBITS, 2)

IQP_single = iqp.IqpSimulator(QUBITS, gates_single_layer, device="lightning.qubit")
IQP_double = iqp.IqpSimulator(QUBITS, gates_two_layer, device="lightning.qubit")

@qml.qnode(dev)
def sample_single(trained_params):
    IQP_single.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def sample_double(trained_params):
    IQP_double.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

In [15]:
with Session(backend=backend, max_time="2h") as session:
    ER18NRes  = sample_double(ER18NParams)
    BP18NDRes = sample_double(BP18NDParams)
    BP18NSRes = sample_single(BP18NSParams)

# Analysis

In [34]:
from utils import metrics
from datasets import utils

In [35]:
def analysis(ground_truth: np.array, samples: np.array, ds_name: str, nodes: int):
    print(f"Number of samples ({ds_name}):", len(samples))
    np.save("./results/samples/{ds_name}.npy", samples)

    graph_ground_truth = [utils.vec_to_graph(vec, nodes) for vec in ground_truth]
    graph_generated = [utils.vec_to_graph(vec, nodes) for vec in samples]

    vec_size = nodes * (nodes - 1) //2
    edge_probability_truth     = np.mean([np.sum(vec)/ vec_size for vec in ground_truth])
    edge_probability_generated = np.mean([np.sum(vec)/ vec_size for vec in samples])

    print("Average edge probability for ground truth: ", edge_probability_truth)
    print("Average edge probability for generated:", edge_probability_generated)
    print("\tDifference: ", np.abs(edge_probability_truth- edge_probability_generated))

    metrics.analyze_model_vs_dataset(graph_ground_truth, graph_generated)

    bipartite_proportion_samples = metrics.bipartite_proportion(samples, nodes)
    bipartite_proportion_truth = metrics.bipartite_proportion(ground_truth, nodes)

    print("Proportion of bipartite graphs in the ground truth", bipartite_proportion_truth * 100, "%")
    print("Proportion of bipartite graphs in the grenerated samples", bipartite_proportion_samples * 100, "%")

    #memorized = metrics.memorized_proportion(graph_ground_truth, graph_generated)
    #print("Memorized samples:", memorized)

    #unique_samples = metrics.unique_samples(graph_generated)
    #print("Proportion of unique samples:", unique_samples[1]* 100, "%")

## 10 Node

In [36]:
analysis(ER10N, ER10NRes, "ER10N", 10)

Number of samples (ER10N): 512
Average edge probability for ground truth:  0.19186666666666669
Average edge probability for generated: 0.19778645833333333
	Difference:  0.005919791666666646

=== Model vs Dataset Analysis ===

1) Basic sizes & uniqueness
   GT graphs       : 500
   Generated graphs: 512
   GT unique-adj  : 500
   Gen unique-adj : 512
   GT unique-iso  : 497
   Gen unique-iso : 424

2) Structural statistics (GT vs Gen)
   Metric               GT       Gen       Δ        Ratio
   Avg edges               8.63     8.90     0.27      1.03
   Avg density             0.19     0.20     0.01      1.03
   Conn. frac              0.15     0.18     0.04      1.24
   #Comp                   2.80     2.77    -0.04      0.99
   Bip frac                0.34     0.43     0.09      1.26
   Bip density             0.31     0.29    -0.02      0.94

3) Memorization & coverage
   Precision (Gen→GT): 34.18%
   Recall    (GT←Gen): 21.20%
   Novel gen        : 337/512 (65.82%)
   GT never seen 

In [37]:
analysis(BP10NS, BP10NSRes, "BP10N_Sparse", 10)

Number of samples (BP10N_Sparse): 512
Average edge probability for ground truth:  0.1770730561428236
Average edge probability for generated: 0.20208333333333334
	Difference:  0.025010277190509744

=== Model vs Dataset Analysis ===

1) Basic sizes & uniqueness
   GT graphs       : 473
   Generated graphs: 512
   GT unique-adj  : 473
   Gen unique-adj : 512
   GT unique-iso  : 473
   Gen unique-iso : 440

2) Structural statistics (GT vs Gen)
   Metric               GT       Gen       Δ        Ratio
   Avg edges               7.97     9.09     1.13      1.14
   Avg density             0.18     0.20     0.03      1.14
   Conn. frac              0.11     0.20     0.09      1.78
   #Comp                   2.85     2.73    -0.12      0.96
   Bip frac                1.00     0.46    -0.54      0.46
   Bip density             0.36     0.33    -0.03      0.92

3) Memorization & coverage
   Precision (Gen→GT): 36.13%
   Recall    (GT←Gen): 26.43%
   Novel gen        : 327/512 (63.87%)
   GT never

In [38]:
analysis(BP10ND, BP10NDRes, "BP10N_Dense", 10)

Number of samples (BP10N_Dense): 512
Average edge probability for ground truth:  0.3469879518072289
Average edge probability for generated: 0.4515190972222222
	Difference:  0.10453114541499331

=== Model vs Dataset Analysis ===

1) Basic sizes & uniqueness
   GT graphs       : 498
   Generated graphs: 512
   GT unique-adj  : 498
   Gen unique-adj : 512
   GT unique-iso  : 498
   Gen unique-iso : 511

2) Structural statistics (GT vs Gen)
   Metric               GT       Gen       Δ        Ratio
   Avg edges              15.61    20.32     4.70      1.30
   Avg density             0.35     0.45     0.10      1.30
   Conn. frac              0.83     0.98     0.15      1.18
   #Comp                   1.32     1.02    -0.29      0.78
   Bip frac                1.00     0.01    -0.99      0.01
   Bip density             0.69     0.67    -0.02      0.97

3) Memorization & coverage
   Precision (Gen→GT):  0.20%
   Recall    (GT←Gen):  0.20%
   Novel gen        : 511/512 (99.80%)
   GT never se

## 14 Node

In [39]:
analysis(ER14N, ER14NRes, "ER14N", 14)

Number of samples (ER14N): 512
Average edge probability for ground truth:  0.15745054945054943
Average edge probability for generated: 0.18264938186813184
	Difference:  0.025198832417582417

=== Model vs Dataset Analysis ===

1) Basic sizes & uniqueness
   GT graphs       : 1000
   Generated graphs: 512
   GT unique-adj  : 1000
   Gen unique-adj : 512
   GT unique-iso  : 999
   Gen unique-iso : 512

2) Structural statistics (GT vs Gen)
   Metric               GT       Gen       Δ        Ratio
   Avg edges              14.33    16.62     2.29      1.16
   Avg density             0.16     0.18     0.03      1.16
   Conn. frac              0.22     0.32     0.11      1.49
   #Comp                   3.25     2.21    -1.04      0.68
   Bip frac                0.28     0.09    -0.19      0.33
   Bip density             0.20     0.25     0.04      1.22

3) Memorization & coverage
   Precision (Gen→GT):  0.20%
   Recall    (GT←Gen):  0.10%
   Novel gen        : 511/512 (99.80%)
   GT never see

In [40]:
analysis(BP14NS, BP14NSRes, "BP14N_Sparse", 14)

Number of samples (BP14N_Sparse): 512
Average edge probability for ground truth:  0.10419128610083385
Average edge probability for generated: 0.14375858516483517
	Difference:  0.03956729906400132

=== Model vs Dataset Analysis ===

1) Basic sizes & uniqueness
   GT graphs       : 995
   Generated graphs: 512
   GT unique-adj  : 995
   Gen unique-adj : 512
   GT unique-iso  : 995
   Gen unique-iso : 505

2) Structural statistics (GT vs Gen)
   Metric               GT       Gen       Δ        Ratio
   Avg edges               9.48    13.08     3.60      1.38
   Avg density             0.10     0.14     0.04      1.38
   Conn. frac              0.01     0.10     0.09      9.36
   #Comp                   5.19     3.48    -1.72      0.67
   Bip frac                1.00     0.34    -0.66      0.34
   Bip density             0.23     0.23    -0.00      0.98

3) Memorization & coverage
   Precision (Gen→GT): 10.74%
   Recall    (GT←Gen):  4.82%
   Novel gen        : 457/512 (89.26%)
   GT never

In [41]:
analysis(BP14ND, BP14NDRes, "BP14N_Dense", 14)

Number of samples (BP14N_Dense): 512
Average edge probability for ground truth:  0.37265448119719474
Average edge probability for generated: 0.4912860576923077
	Difference:  0.11863157649511297

=== Model vs Dataset Analysis ===

1) Basic sizes & uniqueness
   GT graphs       : 995
   Generated graphs: 512
   GT unique-adj  : 995
   Gen unique-adj : 512
   GT unique-iso  : 995
   Gen unique-iso : 512

2) Structural statistics (GT vs Gen)
   Metric               GT       Gen       Δ        Ratio
   Avg edges              33.91    44.71    10.80      1.32
   Avg density             0.37     0.49     0.12      1.32
   Conn. frac              0.93     1.00     0.07      1.08
   #Comp                   1.12     1.00    -0.12      0.89
   Bip frac                1.00     0.00    -1.00      0.00
   Bip density             0.78     0.00    -0.78      0.00

3) Memorization & coverage
   Precision (Gen→GT):  0.00%
   Recall    (GT←Gen):  0.00%
   Novel gen        : 512/512 (100.00%)
   GT never 

## 18 Node

In [42]:
analysis(ER18N, ER18NRes, "ER18N", 18)

Number of samples (ER18N): 512
Average edge probability for ground truth:  0.1496797385620915
Average edge probability for generated: 0.1757174223856209
	Difference:  0.026037683823529417

=== Model vs Dataset Analysis ===

1) Basic sizes & uniqueness
   GT graphs       : 1000
   Generated graphs: 512
   GT unique-adj  : 1000
   Gen unique-adj : 512
   GT unique-iso  : 1000
   Gen unique-iso : 512

2) Structural statistics (GT vs Gen)
   Metric               GT       Gen       Δ        Ratio
   Avg edges              22.90    26.88     3.98      1.17
   Avg density             0.15     0.18     0.03      1.17
   Conn. frac              0.36     0.46     0.10      1.29
   #Comp                   3.28     1.82    -1.46      0.55
   Bip frac                0.18     0.01    -0.16      0.08
   Bip density             0.15     0.20     0.05      1.34

3) Memorization & coverage
   Precision (Gen→GT):  0.00%
   Recall    (GT←Gen):  0.00%
   Novel gen        : 512/512 (100.00%)
   GT never see

In [43]:
analysis(BP18NS, BP18NSRes, "BP18N_Sparse", 18)

Number of samples (BP18N_Sparse): 512
Average edge probability for ground truth:  0.07471537001897532
Average edge probability for generated: 0.09449040032679738
	Difference:  0.01977503030782206

=== Model vs Dataset Analysis ===

1) Basic sizes & uniqueness
   GT graphs       : 992
   Generated graphs: 512
   GT unique-adj  : 992
   Gen unique-adj : 512
   GT unique-iso  : 992
   Gen unique-iso : 511

2) Structural statistics (GT vs Gen)
   Metric               GT       Gen       Δ        Ratio
   Avg edges              11.43    14.46     3.03      1.26
   Avg density             0.07     0.09     0.02      1.26
   Conn. frac              0.01     0.02     0.00      1.29
   #Comp                   7.38     5.48    -1.90      0.74
   Bip frac                1.00     0.39    -0.61      0.39
   Bip density             0.18     0.17    -0.01      0.95

3) Memorization & coverage
   Precision (Gen→GT):  4.88%
   Recall    (GT←Gen):  2.42%
   Novel gen        : 487/512 (95.12%)
   GT never

In [44]:
analysis(BP18ND, BP18NDRes, "BP18N_Dense", 18)

Number of samples (BP18N_Dense): 512
Average edge probability for ground truth:  0.3696521824810326
Average edge probability for generated: 0.3495710784313725
	Difference:  0.020081104049660126

=== Model vs Dataset Analysis ===

1) Basic sizes & uniqueness
   GT graphs       : 995
   Generated graphs: 512
   GT unique-adj  : 995
   Gen unique-adj : 512
   GT unique-iso  : 995
   Gen unique-iso : 512

2) Structural statistics (GT vs Gen)
   Metric               GT       Gen       Δ        Ratio
   Avg edges              56.56    53.48    -3.07      0.95
   Avg density             0.37     0.35    -0.02      0.95
   Conn. frac              0.95     0.95    -0.00      1.00
   #Comp                   1.12     1.05    -0.07      0.94
   Bip frac                1.00     0.00    -1.00      0.00
   Bip density             0.81     0.00    -0.81      0.00

3) Memorization & coverage
   Precision (Gen→GT):  0.00%
   Recall    (GT←Gen):  0.00%
   Novel gen        : 512/512 (100.00%)
   GT never 